In [5]:
# S&P500 2024 data analysis in Excel

import yfinance as yf
import pandas as pd

def get_sp500_tickers():
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    tickers = table[0]['Symbol'].tolist()
    return tickers

sp500_tickers = get_sp500_tickers()

returns_data = []
risk_free_rate = 0.02

# Initialize an Excel writer object to write data
writer = pd.ExcelWriter('sp500_data.xlsx', engine='openpyxl')

# Loop through each ticker and fetch historical data
for ticker in sp500_tickers:
    try:
        # Replace dots in tickers with dashes for compatibility
        adjusted_ticker = ticker.replace('.', '-')
        stock_data = yf.Ticker(adjusted_ticker).history(start='2023-12-29', end='2024-12-31')

        if not stock_data.empty:
            # Remove timezone information from the datetime index
            stock_data.index = stock_data.index.tz_localize(None)
            
            stock_data.to_excel(writer, sheet_name=ticker)  # Save historical data to Excel sheet named after the ticker

            start_price = stock_data.iloc[0]['Close']
            end_price = stock_data.iloc[-1]['Close']
            annual_return = (end_price - start_price) / start_price * 100

            # Calculate daily returns
            stock_data['Daily Return'] = stock_data['Close'].pct_change()
            # Calculate the standard deviation of daily returns (annualized)
            std_dev = stock_data['Daily Return'].std() * (252**0.5)

            # Calculate the Sharpe ratio
            sharpe_ratio = (annual_return / 100 - risk_free_rate) / std_dev

            returns_data.append({'Ticker': ticker, '2024 Return': annual_return, '2024 Std Dev': std_dev, 'Sharpe Ratio': sharpe_ratio})
    except Exception as e:
        print(f"Skipping {ticker} due to error: {e}")

returns_df = pd.DataFrame(returns_data)
returns_df_sorted = returns_df.sort_values(by='2024 Return', ascending=False)

# Save the sorted returns data to a new sheet called 'Output'
returns_df_sorted.to_excel(writer, sheet_name='Output', index=False)

# Save and close the Excel file
writer.close()

print(returns_df_sorted[['Ticker', '2024 Return', '2024 Std Dev', 'Sharpe Ratio']])


    Ticker  2024 Return  2024 Std Dev  Sharpe Ratio
361   PLTR   349.504950      0.643121      5.403413
476    VST   267.350517      0.567212      4.678152
348   NVDA   177.712900      0.525339      3.344752
461    UAL   138.027152      0.447542      3.039428
54    AXON   133.933356      0.449608      2.934409
..     ...          ...           ...           ...
178     EL   -48.390703      0.437598     -1.151531
93      CE   -55.216197      0.390918     -1.463637
250   INTC   -60.028963      0.512579     -1.210134
320   MRNA   -60.402210      0.603295     -1.034356
481    WBA   -62.764718      0.529008     -1.224268

[503 rows x 4 columns]
